# KBMOD Visualization  
  
This notebook demonstrates the basic functionality for working with RawImages and visualizing them.

# Setup kbmod visualization demo
Before importing, make sure you have installed kbmod using `pip install .` in the root directory.  Also be sure you are running with python3 and using the correct notebook kernel.

In [ ]:
# everything we will need for this demo
import kbmod.search as kb
import matplotlib.pyplot as plt
import math
import numpy as np
import os

im_path = "../data/small/"
res_path = "./results"

# Loading data for visualization demo

### A. Load a file for visualization:
The LayeredImage is loaded given the path and filename to the FITS file as well as the PSF for the image. We use a default psf.

In [ ]:
p = kb.PSF(1.0)
im = kb.LayeredImage(im_path + "000000.fits", p)
print(f"Loaded a {im.get_width()} by {im.get_height()} image at time {im.get_obstime()}")

We can visualize the image using matplotlib.

*Note*: The data/demo images contain a single bright object, so the majority of the image should be empty with a single bright spot.

In [ ]:
plt.imshow(im.get_science().image, cmap="gray")

### B. Load a stack of images
A load collection of LayeredImages at different times. 

In [ ]:
files = [im_path + f for f in os.listdir(im_path) if ".fits" in f]
files.sort()

# Create default PSFs for each image.
all_psfs = [p for _ in range(len(files))]

# Load the images.
stack = kb.ImageStack(files, all_psfs)

num_images = stack.img_count()
print(f"Loaded {num_images} images.")

Display each image.

In [ ]:
w = 4
h = math.ceil(num_images / w)

fig, axs = plt.subplots(h, w)
for i in range(h):
    for j in range(w):
        ind = w * i + j
        if ind < num_images:
            axs[i, j].imshow(stack.get_single_image(ind).get_science().image, cmap="gray")

# Create stamps

Stamps are created via the `StampCreator` class. It requires a few pieces of data:
* search_stack - provides the machinery for making predictions on the image (needed to handle the various corrections).
* trajectory - Contains the information about where to place the stamps (the underlying trajectory).
* stamp_radius - The radius in pixels.

In [ ]:
# Create the trajectory with a given parameters and then the trajectory result.
trj = kb.Trajectory()
trj.x = 11
trj.y = 27
trj.vx = 16.0
trj.vy = 3.3

# Create the stamps around this trajectory.
stamps = kb.StampCreator.get_stamps(stack, trj, 20)

Now we can display the stamps around each predicted object position.

In [ ]:
fig, axs = plt.subplots(h, w)
for i in range(h):
    for j in range(w):
        ind = w * i + j
        if ind < num_images:
            axs[i, j].imshow(stamps[ind].image, cmap="gray")

We can also compute the sum, mean, or median stamp. The stamp functions take a list of bools corresponding to whether each time is valid. These can be extracted from the result data. You can use an empty array (as we do below) to build the stamp out of all valid times.

In [ ]:
fig, axs = plt.subplots(1, 3)

# The coadds requires a vector of which indices to use.
indices = [True] * stack.img_count()

axs[0].imshow(kb.StampCreator.get_summed_stamp(stack, trj, 10, indices).image, cmap="gray")
axs[0].set_title("Summed")

axs[1].imshow(kb.StampCreator.get_mean_stamp(stack, trj, 10, indices).image, cmap="gray")
axs[1].set_title("Mean")

axs[2].imshow(kb.StampCreator.get_median_stamp(stack, trj, 10, indices).image, cmap="gray")
axs[2].set_title("Median")